# r.windfetch

This notebook runs the example from r.windfetch's manual page and visualizes the output.

## Setup

We will be using the NC SPM sample location.

In [ ]:
import os
import json
import subprocess
import sys
import matplotlib.pyplot as plt
import numpy as np

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata/nc_basic_spm_grass7/user1")

## Create example from the tool's manual page

```
r.mapcalc "land = if (isnull(lakes), 1, 0)"
r.windfetch input=land format=json coordinates=635659,223234
```

In [ ]:
gs.run_command("g.region", n=224140, s=222330, w=634500, e=636400, res=10)
gs.mapcalc("land = if (isnull(lakes), 1, 0)")
point = (635659, 223234)
data = gs.parse_command("r.windfetch", input="land", format="json", coordinates=point)
gs.write_command(
    "v.in.ascii", input="-", output="point", stdin=f"{point[0]}|{point[1]}"
)

In [ ]:
r_windfetch_map = gj.Map(use_region=True)
r_windfetch_map.d_rast(map="elevation")
r_windfetch_map.d_rast(map="lakes")
r_windfetch_map.d_vect(map="point", icon="basic/circle", size=15, fill_color="black")
r_windfetch_map.show()

In [ ]:
directions = np.deg2rad(
    data[0]["directions"]
)  # Convert degrees to radians for polar plot
fetch = data[0]["fetch"]
from PIL import Image

# Open the PNG image and the plot
png_image = Image.open(r_windfetch_map.filename)

plot_image = plt.figure(figsize=(5, 5))

# Plot your polar plot
ax = plt.subplot(111, polar=True)
ax.plot(directions, fetch, marker="o", linestyle="-")
ax.fill(directions, fetch, alpha=0.3)
ax.set_theta_zero_location("E")
ax.set_theta_direction(1)
ax.set_title("Wind fetch", va="bottom")

# Save the plot as a temporary image
plot_image.savefig("temp_plot.png")

# Open the saved plot image
plot_image = Image.open("temp_plot.png")

# Determine the scale factor for resizing
max_width = 800
scale_factor = max_width / float(png_image.width + plot_image.width)

# Resize the images
png_image_resized = png_image.resize(
    (int(png_image.width * scale_factor), int(png_image.height * scale_factor))
)
plot_image_resized = plot_image.resize(
    (int(plot_image.width * scale_factor), int(plot_image.height * scale_factor))
)

# Determine the size of the combined image
max_height = max(png_image_resized.height, plot_image_resized.height)
combined_width = png_image_resized.width + plot_image_resized.width

# Create a new blank image with the size of the combined image
combined_image = Image.new("RGB", (combined_width, max_height), color="white")

# Paste the PNG image on the left side
combined_image.paste(png_image_resized, (0, 0))

# Paste the plot image on the right side
combined_image.paste(plot_image_resized, (png_image_resized.width, 0))

# Show the combined image
combined_image.show()

# Save the combined image
combined_image.save("r_windfetch.png")